In [1]:
import sklearn as sk
import pandas as pd
import numpy as np
from os import getenv
from datetime import datetime
from amb_sdk.sdk import DarwinSdk
import matplotlib.pyplot as plt
%matplotlib inline

pd.set_option('display.max_rows', None)

In [2]:
ds = DarwinSdk()
ds.set_url('https://amb-demo-api.sparkcognition.com/v1/')

status, msg = ds.auth_login_user('nathanalvarezolson@utexas.edu','Hmy35qPZM4')
if not status:
    print(msg)
status

True

In [3]:
cycle_df = pd.read_csv('./Austin_B-Cycle_Trips.csv')
weather_df = pd.read_csv('./austin_weather.csv')
stations_df = pd.read_csv('austin_bikeshare_stations.csv')

In [4]:
stations_df.head()

,latitude,location,longitude,name,station_id,status
0,30.27041,(30.27041 -97.75046),-97.75046,West & 6th St.,2537,active
1,30.26452,(30.26452 -97.7712),-97.77120,Barton Springs Pool,2572,active
2,30.27595,(30.27595 -97.74739),-97.74739,ACC - Rio Grande & 12th,2545,closed
3,30.28480,(30.2848 -97.72756),-97.72756,Red River & LBJ Library,1004,closed
4,30.26694,(30.26694 -97.74939),-97.74939,Nueces @ 3rd,1008,moved


In [5]:
# May have to uncomment out the following line in order to run. Installing distance
#!pip install geopy
from geopy import distance

In [6]:
# Doing a Cartesian product on the stations dataframe, so we may have pairs of stations
from pandas import merge

stations_df['merge_key'] = 0
pairs_of_stations_df = stations_df.merge(stations_df, how='left', on = 'merge_key')

In [7]:
# Calculating distance with the geopy.distance method

pairs_of_stations_df['distance_miles'] = 1.1

for i in range(len(pairs_of_stations_df)):
    location1 = (pairs_of_stations_df.loc[i, 'latitude_x'] , pairs_of_stations_df.loc[i, 'longitude_x'] )
    location2 = (pairs_of_stations_df.loc[i, 'latitude_y'] , pairs_of_stations_df.loc[i, 'longitude_y'] )
    pairs_of_stations_df.loc[i, 'distance_miles'] = distance.distance(location1, location2).miles

In [8]:
# Dropping the irrelevant columns
pairs_of_stations_df.drop(['latitude_x', 'location_x', 'longitude_x', 'status_x', 'merge_key',
                           'latitude_y', 'location_y', 'longitude_y', 'status_y'],1, inplace=True)

print(pairs_of_stations_df.head())

           name_x  station_id_x                   name_y  station_id_y  \
0  West & 6th St.          2537           West & 6th St.          2537   
1  West & 6th St.          2537      Barton Springs Pool          2572   
2  West & 6th St.          2537  ACC - Rio Grande & 12th          2545   
3  West & 6th St.          2537  Red River & LBJ Library          1004   
4  West & 6th St.          2537             Nueces @ 3rd          1008   

   distance_miles  
0        0.000000  
1        1.304777  
2        0.423463  
3        1.690263  
4        0.247439  


In [9]:
pairs_of_stations_df.to_csv('station_pairs_dist.csv', index=False)